In [1]:
from typing_extensions import TypedDict, Annotated, List, Literal
from langchain_core.messages import AnyMessage, AIMessage, HumanMessage
from langgraph.graph import add_messages
from typing import Dict, List, Optional, Union, Literal
from pydantic import BaseModel, Field
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image
import operator

In [2]:
class DocumentSection(BaseModel):
    title: str = Field(description="The title of the section")
    content: str = Field(description="The content of the section")

class DocumentState(BaseModel):
    document_type: Literal["functional", "technical"] = "functional"
    functional_status: Literal["in_progress", "pending_approval", "feedback", "completed"] = "in_progress"
    technical_status: Literal["in_progress", "pending_approval", "feedback", "completed"] = "in_progress"
    revised_count : int = 0
    version: float = 1.0
    functional_documents: List[DocumentSection] = []
    technical_documents: List[DocumentSection] = []
    messages: Annotated[list, add_messages] = []

In [3]:
## Design documents
def create_design_documents(state: DocumentState) -> DocumentState:
    print("In create_design_documents")

    doc_type = state.document_type.lower()
    is_functional = doc_type == "functional"

    # Define common sections
    document1 = DocumentSection(
        title="INTRODUCTION" if is_functional else "INTRODUCTION & PURPOSE",
        content="This document defines the functional requirements for the Password Reset Feature of the User Management System."
        if is_functional else
        "This document outlines the technical design for the Password Reset functionality as defined in the Functional Specification Document (FSD)."
    )

    document2 = DocumentSection(
        title="BUSINESS CONTEXT",
        content="The business needs a secure mechanism for users to recover access to their accounts without compromising security, improving customer satisfaction and retention.",
    )

    documents = [document1, document2]

    return {
        f"{doc_type}_documents": documents,
        f"{doc_type}_status": 'pending_approval',
        "messages": AIMessage(
            content=f"Please review above {doc_type} design document and provide feedback or type 'Approved' if you're satisfied."
        ),
        "revised_count": 0,
    }

def design_documents_review(state: DocumentState) -> DocumentState:
    print("In design_documents_review")
    message_content = state.messages[-1].content.lower().strip()
    print("user feedback:", message_content)

    approved = message_content == "approved"
    doc_type = state.document_type

    response = {
        "messages": AIMessage(
            content="Great! Your design documents have been finalized. You can now proceed with your design process."
            if approved else
            "I've received your feedback. I'll revise the design documents accordingly."
        ),
        f"{doc_type}_status": "completed" if approved else "feedback"
    }

    return response


def should_revise_design_documents(state: DocumentState) -> Literal["feedback", "approved"]:
    print("In should_revise_design_documents")
    if state.document_type == "functional":
        return "feedback" if state.functional_status == "feedback" else "approved"
    
    if state.document_type == "technical":
        return "feedback" if state.technical_status == "feedback" else "approved"
    
    return "approved"  # Default fallback


def revised_design_documents(state: DocumentState) -> DocumentState:
    print("In revised_design_documents")
    print("state :", state)

    doc_type = None
    if state.functional_status == 'feedback':
        doc_type = "functional"
    elif state.technical_status == 'feedback':
        doc_type = "technical"

    if not doc_type:
        return state  # No feedback status, return state unchanged

    revised_count = state.revised_count + 1
    print("revised_count :", revised_count)

    if revised_count == 3:
        return {
            "messages": AIMessage(
                content="Design documents have been revision maxed out. Please review the above documents and continue with the next step."
            ),
            f"{doc_type}_status": "completed"
        }

    # Define document details dynamically
    document_data = {
        "functional": DocumentSection(
                title = f"Revised Functional Design Document : FUNCTIONAL REQUIREMENTS",
                content = """
                FR-1: The system shall provide a "Forgot Password" option on the login page.  
                FR-2: The system shall send a time-limited reset link to the user's registered email address.  
                FR-3: The system shall validate new passwords against password policy rules.  
                FR-4: The system shall notify users of successful password reset and redirect them to the login page.
            """),
        "technical": DocumentSection(
                title = f"Revised Technical Design Document : MODULES & COMPONENTS DESIGN",
                content = """
                - **Auth API Module:** Exposes RESTful endpoints for password reset request and confirmation  
                - **Token Service Module:** Generates, hashes, stores, and validates password reset tokens  
                - **Notification Service:** Sends password reset links securely via AWS SES  
                - **Audit Logging Module:** Captures all user-initiated reset activities for security auditing
            """,
            )
    }

    doc_info = document_data[doc_type]
    revised_document = DocumentSection(title=doc_info.title, content=doc_info.content)

    return {
        f"{doc_type}_documents": [revised_document],
        "messages": AIMessage(
            content=f"Please review above revised {doc_type} design documents and provide additional feedback or type 'Approved' if you're satisfied."
        ),
        f"{doc_type}_status": "pending_approval",
        "revised_count": revised_count
    }


In [4]:
## Design documents 
from langgraph.graph import StateGraph, START, END
document_graph = StateGraph(DocumentState)

document_graph.add_node("create_design_documents", create_design_documents)
document_graph.add_node("design_documents_review", design_documents_review)
document_graph.add_node("revised_design_documents", revised_design_documents)

## Design documents 
document_graph.add_edge(START, "create_design_documents")
document_graph.add_edge("create_design_documents", "design_documents_review")
document_graph.add_conditional_edges(
    "design_documents_review",
    should_revise_design_documents,
    {
        "feedback" : "revised_design_documents",
        "approved" : END
    }
)
document_graph.add_edge("revised_design_documents", "design_documents_review")

memory = MemorySaver()
document_workflow = document_graph.compile(checkpointer=memory, interrupt_before=['design_documents_review'])

In [6]:
display(Image(document_workflow.get_graph().draw_mermaid_png()))

ConnectionError: HTTPSConnectionPool(host='mermaid.ink', port=443): Max retries exceeded with url: /img/LS0tCmNvbmZpZzoKICBmbG93Y2hhcnQ6CiAgICBjdXJ2ZTogbGluZWFyCi0tLQpncmFwaCBURDsKCV9fc3RhcnRfXyhbPHA+X19zdGFydF9fPC9wPl0pOjo6Zmlyc3QKCWNyZWF0ZV9kZXNpZ25fZG9jdW1lbnRzKGNyZWF0ZV9kZXNpZ25fZG9jdW1lbnRzKQoJZGVzaWduX2RvY3VtZW50c19yZXZpZXcoZGVzaWduX2RvY3VtZW50c19yZXZpZXc8aHIvPjxzbWFsbD48ZW0+X19pbnRlcnJ1cHQgPSBiZWZvcmU8L2VtPjwvc21hbGw+KQoJcmV2aXNlZF9kZXNpZ25fZG9jdW1lbnRzKHJldmlzZWRfZGVzaWduX2RvY3VtZW50cykKCV9fZW5kX18oWzxwPl9fZW5kX188L3A+XSk6OjpsYXN0CglfX3N0YXJ0X18gLS0+IGNyZWF0ZV9kZXNpZ25fZG9jdW1lbnRzOwoJY3JlYXRlX2Rlc2lnbl9kb2N1bWVudHMgLS0+IGRlc2lnbl9kb2N1bWVudHNfcmV2aWV3OwoJcmV2aXNlZF9kZXNpZ25fZG9jdW1lbnRzIC0tPiBkZXNpZ25fZG9jdW1lbnRzX3JldmlldzsKCWRlc2lnbl9kb2N1bWVudHNfcmV2aWV3IC0uICZuYnNwO2ZlZWRiYWNrJm5ic3A7IC4tPiByZXZpc2VkX2Rlc2lnbl9kb2N1bWVudHM7CglkZXNpZ25fZG9jdW1lbnRzX3JldmlldyAtLiAmbmJzcDthcHByb3ZlZCZuYnNwOyAuLT4gX19lbmRfXzsKCWNsYXNzRGVmIGRlZmF1bHQgZmlsbDojZjJmMGZmLGxpbmUtaGVpZ2h0OjEuMgoJY2xh

In [7]:

initial_document_state = {
    "functional_documents" : [],
    "technical_documents" : [],
    "document_type" : "functional",
    "messages": [],
    "functional_status": "in_progress",
    "revised_count" : 0,
    "version" : 1.0
}

# Thread
thread = {"configurable": {"thread_id": "20"}}

for event in document_workflow.stream(initial_document_state, thread, stream_mode="values"):
    print(event)

{'functional_documents': [], 'technical_documents': [], 'messages': [], 'document_type': 'functional', 'functional_status': 'in_progress', 'revised_count': 0, 'version': 1.0}
In create_design_documents
{'functional_documents': [DocumentSection(title='INTRODUCTION', content='This document defines the functional requirements for the Password Reset Feature of the User Management System.'), DocumentSection(title='BUSINESS CONTEXT', content='The business needs a secure mechanism for users to recover access to their accounts without compromising security, improving customer satisfaction and retention.')], 'technical_documents': [], 'messages': [AIMessage(content="Please review above functional design document and provide feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='d2b29f4a-e224-42c4-9d89-54dc4859f117')], 'document_type': 'functional', 'functional_status': 'pending_approval', 'revised_count': 0, 'version': 1.0}


Failed to get info from https://api.smith.langchain.com: LangSmithConnectionError('Connection error caused failure to GET /info in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError(\'HTTPSConnectionPool(host=\\\'api.smith.langchain.com\\\', port=443): Max retries exceeded with url: /info (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x105d25820>: Failed to resolve \\\'api.smith.langchain.com\\\' ([Errno 8] nodename nor servname provided, or not known)"))\'))\nContent-Length: None\nAPI Key: lsv2_********************************************a7')
Failed to batch ingest runs: langsmith.utils.LangSmithConnectionError: Connection error caused failure to POST https://api.smith.langchain.com/runs/batch in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/batch (Caused by NameResolutionError(

In [ ]:
{'functional_documents': 
 [DocumentSection(title='INTRODUCTION', content='This document defines the functional requirements for the Password Reset Feature of the User Management System.'), 
  DocumentSection(title='BUSINESS CONTEXT', content='The business needs a secure mechanism for users to recover access to their accounts without compromising security, improving customer satisfaction and retention.')], 'technical_documents': [], 
  'messages': [AIMessage(content="Please review above functional design document and provide feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='d2b29f4a-e224-42c4-9d89-54dc4859f117')], 
  'document_type': 'functional', 'functional_status': 'pending_approval', 'revised_count': 0, 'version': 1.0}

In [8]:
document_state = document_workflow.get_state(thread) 
document_state.next

('design_documents_review',)

In [9]:

updated_state = document_workflow.update_state(thread, { "messages" : HumanMessage(content='Add signal player game')})
# document_workflow.update_state(thread, { "messages" : HumanMessage(content='Approved')})

# Continue the graph execution
for event in document_workflow.stream(None, thread, stream_mode="values"):
    print(event)

{'functional_documents': [DocumentSection(title='INTRODUCTION', content='This document defines the functional requirements for the Password Reset Feature of the User Management System.'), DocumentSection(title='BUSINESS CONTEXT', content='The business needs a secure mechanism for users to recover access to their accounts without compromising security, improving customer satisfaction and retention.')], 'technical_documents': [], 'messages': [AIMessage(content="Please review above functional design document and provide feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='d2b29f4a-e224-42c4-9d89-54dc4859f117'), HumanMessage(content='Add signal player game', additional_kwargs={}, response_metadata={}, id='dd156dd0-867b-4b3f-9abe-fc8d9f0b786b')], 'document_type': 'functional', 'functional_status': 'pending_approval', 'revised_count': 0, 'version': 1.0}
In design_documents_review
user feedback: add signal player game
In should_revise_design_docum

In [ ]:
{'functional_documents': [DocumentSection(title='Revised Functional Design Document : FUNCTIONAL REQUIREMENTS', content='\n                FR-1: The system shall provide a "Forgot Password" option on the login page.  \n                FR-2: The system shall send a time-limited reset link to the user\'s registered email address.  \n                FR-3: The system shall validate new passwords against password policy rules.  \n                FR-4: The system shall notify users of successful password reset and redirect them to the login page.\n            ')], 'technical_documents': [], 'messages': [AIMessage(content="Please review above functional design document and provide feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='d2b29f4a-e224-42c4-9d89-54dc4859f117'), HumanMessage(content='Add signal player game', additional_kwargs={}, response_metadata={}, id='dd156dd0-867b-4b3f-9abe-fc8d9f0b786b'), AIMessage(content="I've received your feedback. I'll revise the design documents accordingly.", additional_kwargs={}, response_metadata={}, id='48907e7d-b5f0-42bb-b8f7-52e01f39e6d9'), AIMessage(content="Please review above revised functional design documents and provide additional feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='41366e89-8c68-4d43-9124-7f33167daa97')], 'document_type': 'functional', 'functional_status': 'pending_approval', 'revised_count': 1, 'version': 1.0}

## technical document

In [10]:

initial_document_state = {
    "technical_documents" : [],
    "document_type" : "technical",
    "technical_status" : "in_progress",
    "revised_count" : 0
}

# Thread
# thread = {"configurable": {"thread_id": "20"}}

for event in document_workflow.stream(initial_document_state, thread, stream_mode="values"):
    print(event)

{'functional_documents': [DocumentSection(title='Revised Functional Design Document : FUNCTIONAL REQUIREMENTS', content='\n                FR-1: The system shall provide a "Forgot Password" option on the login page.  \n                FR-2: The system shall send a time-limited reset link to the user\'s registered email address.  \n                FR-3: The system shall validate new passwords against password policy rules.  \n                FR-4: The system shall notify users of successful password reset and redirect them to the login page.\n            ')], 'technical_documents': [], 'messages': [AIMessage(content="Please review above functional design document and provide feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='d2b29f4a-e224-42c4-9d89-54dc4859f117'), HumanMessage(content='Add signal player game', additional_kwargs={}, response_metadata={}, id='dd156dd0-867b-4b3f-9abe-fc8d9f0b786b'), AIMessage(content="I've received your feedb

In [11]:
{'functional_documents': [DocumentSection(title='Revised Functional Design Document : FUNCTIONAL REQUIREMENTS', content='\n                FR-1: The system shall provide a "Forgot Password" option on the login page.  \n                FR-2: The system shall send a time-limited reset link to the user\'s registered email address.  \n                FR-3: The system shall validate new passwords against password policy rules.  \n                FR-4: The system shall notify users of successful password reset and redirect them to the login page.\n            ')], 'technical_documents': [DocumentSection(title='INTRODUCTION & PURPOSE', content='This document outlines the technical design for the Password Reset functionality as defined in the Functional Specification Document (FSD).'), DocumentSection(title='BUSINESS CONTEXT', content='The business needs a secure mechanism for users to recover access to their accounts without compromising security, improving customer satisfaction and retention.')], 'messages': [AIMessage(content="Please review above functional design document and provide feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='d2b29f4a-e224-42c4-9d89-54dc4859f117'), HumanMessage(content='Add signal player game', additional_kwargs={}, response_metadata={}, id='dd156dd0-867b-4b3f-9abe-fc8d9f0b786b'), AIMessage(content="I've received your feedback. I'll revise the design documents accordingly.", additional_kwargs={}, response_metadata={}, id='48907e7d-b5f0-42bb-b8f7-52e01f39e6d9'), AIMessage(content="Please review above revised functional design documents and provide additional feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='41366e89-8c68-4d43-9124-7f33167daa97'), AIMessage(content="Please review above technical design document and provide feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='7c000f71-217d-47f3-8bf1-607aa6227d90')], 'document_type': 'technical', 'functional_status': 'pending_approval', 'technical_status': 'pending_approval', 'revised_count': 0, 'version': 1.0}


{'functional_documents': [DocumentSection(title='Revised Functional Design Document : FUNCTIONAL REQUIREMENTS', content='\n                FR-1: The system shall provide a "Forgot Password" option on the login page.  \n                FR-2: The system shall send a time-limited reset link to the user\'s registered email address.  \n                FR-3: The system shall validate new passwords against password policy rules.  \n                FR-4: The system shall notify users of successful password reset and redirect them to the login page.\n            ')],
 'technical_documents': [DocumentSection(title='INTRODUCTION & PURPOSE', content='This document outlines the technical design for the Password Reset functionality as defined in the Functional Specification Document (FSD).'),
  DocumentSection(title='BUSINESS CONTEXT', content='The business needs a secure mechanism for users to recover access to their accounts without compromising security, improving customer satisfaction and retent

In [12]:
document_state = document_workflow.get_state(thread) 
document_state.next

('design_documents_review',)

In [13]:
updated_state = document_workflow.update_state(thread, { "messages" : HumanMessage(content='correct the document')})
# document_workflow.update_state(thread, { "messages" : HumanMessage(content='Approved')})

# Continue the graph execution
for event in document_workflow.stream(None, thread, stream_mode="values"):
    print(event)

{'functional_documents': [DocumentSection(title='Revised Functional Design Document : FUNCTIONAL REQUIREMENTS', content='\n                FR-1: The system shall provide a "Forgot Password" option on the login page.  \n                FR-2: The system shall send a time-limited reset link to the user\'s registered email address.  \n                FR-3: The system shall validate new passwords against password policy rules.  \n                FR-4: The system shall notify users of successful password reset and redirect them to the login page.\n            ')], 'technical_documents': [DocumentSection(title='INTRODUCTION & PURPOSE', content='This document outlines the technical design for the Password Reset functionality as defined in the Functional Specification Document (FSD).'), DocumentSection(title='BUSINESS CONTEXT', content='The business needs a secure mechanism for users to recover access to their accounts without compromising security, improving customer satisfaction and retention

In [14]:
{'functional_documents': [DocumentSection(title='Revised Functional Design Document : FUNCTIONAL REQUIREMENTS', content='\n                FR-1: The system shall provide a "Forgot Password" option on the login page.  \n                FR-2: The system shall send a time-limited reset link to the user\'s registered email address.  \n                FR-3: The system shall validate new passwords against password policy rules.  \n                FR-4: The system shall notify users of successful password reset and redirect them to the login page.\n            ')], 'technical_documents': [DocumentSection(title='Revised Technical Design Document : MODULES & COMPONENTS DESIGN', content='\n                - **Auth API Module:** Exposes RESTful endpoints for password reset request and confirmation  \n                - **Token Service Module:** Generates, hashes, stores, and validates password reset tokens  \n                - **Notification Service:** Sends password reset links securely via AWS SES  \n                - **Audit Logging Module:** Captures all user-initiated reset activities for security auditing\n            ')], 'messages': [AIMessage(content="Please review above functional design document and provide feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='d2b29f4a-e224-42c4-9d89-54dc4859f117'), HumanMessage(content='Add signal player game', additional_kwargs={}, response_metadata={}, id='dd156dd0-867b-4b3f-9abe-fc8d9f0b786b'), AIMessage(content="I've received your feedback. I'll revise the design documents accordingly.", additional_kwargs={}, response_metadata={}, id='48907e7d-b5f0-42bb-b8f7-52e01f39e6d9'), AIMessage(content="Please review above revised functional design documents and provide additional feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='41366e89-8c68-4d43-9124-7f33167daa97'), AIMessage(content="Please review above technical design document and provide feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='7c000f71-217d-47f3-8bf1-607aa6227d90'), HumanMessage(content='correct the document', additional_kwargs={}, response_metadata={}, id='bba098a1-d32e-43c4-901b-01a841209719'), AIMessage(content="I've received your feedback. I'll revise the design documents accordingly.", additional_kwargs={}, response_metadata={}, id='2b686a83-f979-40d6-9e71-20c452d8aa43'), AIMessage(content="Please review above revised technical design documents and provide additional feedback or type 'Approved' if you're satisfied.", additional_kwargs={}, response_metadata={}, id='ba989393-0f93-4095-906b-6d83f895632c')], 'document_type': 'technical', 'functional_status': 'pending_approval', 'technical_status': 'pending_approval', 'revised_count': 1, 'version': 1.0}

{'functional_documents': [DocumentSection(title='Revised Functional Design Document : FUNCTIONAL REQUIREMENTS', content='\n                FR-1: The system shall provide a "Forgot Password" option on the login page.  \n                FR-2: The system shall send a time-limited reset link to the user\'s registered email address.  \n                FR-3: The system shall validate new passwords against password policy rules.  \n                FR-4: The system shall notify users of successful password reset and redirect them to the login page.\n            ')],
 'technical_documents': [DocumentSection(title='Revised Technical Design Document : MODULES & COMPONENTS DESIGN', content='\n                - **Auth API Module:** Exposes RESTful endpoints for password reset request and confirmation  \n                - **Token Service Module:** Generates, hashes, stores, and validates password reset tokens  \n                - **Notification Service:** Sends password reset links securely via AWS 